<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [6]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 

DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [7]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

OperationalError: connection to server at "84.201.134.129", port 5432 failed: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.


# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [ ]:
# текст запроса

query_3_1 = f'''
    SELECT COUNT(v.id)
    FROM public.vacancies as v
    '''

In [ ]:
# результат запроса

df = pd.read_sql_query(query_3_1, connection);
print(f'Количество вакансий в нашей базе: {df['count'][0]}')

Количество вакансий в нашей базе: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [ ]:
# текст запроса

query_3_2 = f'''
    SELECT COUNT(e.id)
    FROM public.employers as e
    '''

In [ ]:
# результат запроса

df = pd.read_sql_query(query_3_2, connection)
print(f'Количество работодателей в нашей базе: {df['count'][0]}')

Количество работодателей в нашей базе: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [ ]:
# текст запроса

query_3_3 = f'''
    SELECT COUNT(a.id)
    FROM public.areas as a
    '''
    

In [ ]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
print(f'Количество регионов в нашей базе: {df['count'][0]}')


Количество регионов в нашей базе: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [ ]:
# текст запроса

query_3_4 = f'''
    SELECT COUNT(i.id)
    FROM public.industries as i
    '''

In [ ]:
# результат запроса

df = pd.read_sql_query(query_3_4, connection)
print(f'Количество сфер деятельности в нашей базе: {df['count'][0]}')

Количество сфер деятельности в нашей базе: 294


***

### Выводы по предварительному анализу данных

Наша база состоит из:

* 49197 вакансий; 
* 23501 работодателя; 
* Вакансии размещены в 1362 разных регоинах;
* и затрагивают 294 сферы деятельности.

В среднем на работодателя приходится по 2.09 вакансии



# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [ ]:
# текст запроса

limit = 5

query_4_1 = f'''
    SELECT 
        COUNT(v.id) as vacancies_count,
        a.name as area
    FROM public.vacancies as v
    JOIN public.areas as a on v.area_id = a.id
    GROUP BY a.name
    ORDER BY vacancies_count DESC
    LIMIT {limit}
    '''

In [ ]:
# результат запроса

df = pd.read_sql_query(query_4_1, connection)
df

,vacancies_count,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [ ]:
# текст запроса

query_4_2 = f'''
    SELECT 
        COUNT(v.id)
    FROM public.vacancies as v
    WHERE v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL
    '''

In [ ]:
# результат запроса

df = pd.read_sql_query(query_4_2, connection)
print(f'Количество вакансий, где заполнено хотя бы одно из двух полей с зарплатой: {df['count'][0]}')

Количество вакансий, где заполнено хотя бы одно из двух полей с зарплатой: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [ ]:
# текст запроса

query_4_3 = f'''
    SELECT 
        ROUND(AVG(v.salary_from)) avg_min_salary,
        ROUND(AVG(v.salary_to)) avg_max_salary
    FROM public.vacancies as v
    WHERE v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL
    '''

In [ ]:
# результат 

df = pd.read_sql_query(query_4_3, connection)
print(f'Средние значения для нижней границы зарплатной вилки: {df['avg_min_salary'][0]}')
print(f'Средние значения для верхней границы зарплатной вилки: {df['avg_max_salary'][0]}')

Средние значения для нижней границы зарплатной вилки: 71065.0
Средние значения для верхней границы зарплатной вилки: 110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [ ]:
# текст запроса

limit = 1
offset = 1

query_4_4 = f'''
    SELECT
        v.schedule || ' - ' || v.employment as type,
        COUNT(*) as cnt
    FROM public.vacancies as v
    GROUP BY v.schedule || ' - ' || v.employment
    ORDER BY cnt DESC
    OFFSET {offset}
    LIMIT {limit}
    '''


In [ ]:
# результат запроса

df = pd.read_sql_query(query_4_4, connection)
df


,type,cnt
0,Удаленная работа - Полная занятость,7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [ ]:
# текст запроса

limit = 1
offset = 1

query_4_5 = f'''
    SELECT
        COUNT(*) as cnt,
        v.experience
    FROM public.vacancies as v
    GROUP BY v.experience
    ORDER BY cnt ASC
    '''


In [ ]:
# результат запроса

df = pd.read_sql_query(query_4_5, connection)
df

,cnt,experience
0,1337,Более 6 лет
1,7197,Нет опыта
2,14511,От 3 до 6 лет
3,26152,От 1 года до 3 лет


***

### Выводы по детальному анализу вакансий

* Наибольшое количество вакансий размещается в Москве;
* Видна тенденция, что все чаще требуются работники на удаленную работу;
* Только у 50% вакансий указаны данные о зарплате
* 55% вакансией требуют хотя бы минимальный опыт, а в 15% не требуетс опыт вообще
* На 35% средняя верхняя граница больше нижней границу в зарплатных вилках

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
# текст запроса

limit = 1
offset = 4

query_5_1 = f'''
    (
        SELECT
            COUNT(*) as cnt,
            e.name
        FROM public.vacancies as v
        JOIN public.employers as e on v.employer_id = e.id
        GROUP BY e.name
        ORDER BY cnt DESC
        LIMIT {limit}
    )
    UNION
    (
        SELECT
            COUNT(*) as cnt,
            e.name
        FROM public.vacancies as v
        JOIN public.employers as e on v.employer_id = e.id
        GROUP BY e.name
        ORDER BY cnt DESC
        OFFSET {offset}
        LIMIT {limit}
    )
    ORDER BY cnt DESC
    '''

In [ ]:
# результат запроса

df = pd.read_sql_query(query_5_1, connection)
df

,cnt,name
0,1933,Яндекс
1,331,Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
# текст запроса

limit = 1

query_5_2 = f'''
        SELECT
            a.name as name,
            COUNT(v.id) as vacancies_count,
            COUNT(DISTINCT e.id) as employers_count
        FROM public.areas as a
        LEFT JOIN public.employers as e on a.id = e.area
        LEFT JOIN public.vacancies as v on a.id = v.area_id
        WHERE v.id is null
        GROUP BY a.name
        ORDER BY employers_count DESC
        LIMIT {limit}
        '''

In [ ]:
# результат запроса

df = pd.read_sql_query(query_5_2, connection)
print(f'В регионе {df['name'][0]} наибольшое количество работодателей ({df['employers_count'][0]}), которые не разместили вакансии')

В регионе Россия наибольшое количество работодателей (410), которые не разместили вакансии


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
# текст запроса

limit = 1

query_5_3 = f'''
        SELECT
            e.name as name,
            COUNT(distinct a.id) as area_count
        FROM public.vacancies as v
        JOIN public.employers as e on v.employer_id = e.id
        JOIN public.areas as a on a.id = v.area_id
        GROUP BY e.name
        ORDER BY area_count DESC
        LIMIT {limit}
        '''

In [ ]:
# результат запроса

df = pd.read_sql_query(query_5_3, connection)
print(f'{df['name'][0]} является компанией, разместившей наибольшое количество вакансией в разных регионах: {df['area_count'][0]}')

Яндекс является компанией, разместившей наибольшое количество вакансией в разных регионах: 181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [ ]:
# текст запроса

query_5_4 = f'''
        SELECT
            COUNT(distinct e.id) as employers_count
        FROM public.employers as e
        FULL OUTER JOIN public.employers_industries as ei on e.id = ei.employer_id
        WHERE ei.industry_id is null
        '''

In [ ]:
df = pd.read_sql_query(query_5_4, connection)
print(f'{df['employers_count'][0]} компаний, не предоставляют информацию, в какой сфере дейтельности они работают')

8419 компаний, не предоставляют информацию, в какой сфере дейтельности они работают


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [ ]:
# текст запроса

limit = 1
offset = 2

query_5_5 = f'''
        SELECT
            e.name as employer_name,
            COUNT(ei.industry_id)
        FROM public.employers as e
        FULL OUTER JOIN public.employers_industries as ei on e.id = ei.employer_id
        GROUP BY e.name
        HAVING COUNT(ei.industry_id) = 4
        ORDER BY 1 ASC
        OFFSET {offset}
        LIMIT {limit}
        '''

In [ ]:
# результат запроса

df = pd.read_sql_query(query_5_5, connection)
df

,employer_name,count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [ ]:
# текст запроса

query_5_6 = f'''
        SELECT
            COUNT(ei.employer_id)
        FROM public.employers_industries as ei
        JOIN public.industries as i on ei.industry_id = i.id
        WHERE i.name LIKE 'Разработка программного обеспечения%'
        '''

In [ ]:
# результат запроса

df = pd.read_sql_query(query_5_6, connection)
print(f'{df['count'][0]} компаний, занимаются разработкой программного обеспечения')

3553 компаний, занимаются разработкой программного обеспечения


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [ ]:
# код для получения списка городов-милионников

import requests 
from bs4 import BeautifulSoup 

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'

# Получаем содержимое страницы
response = requests.get(url)
response.encoding = 'utf-8'  # Устанавливаем кодировку

# Парсим HTML-код
page = BeautifulSoup(response.text, 'html.parser')

# Находим таблицу с городами-миллионерами
table = page.find('table', class_='standard')

# Извлекаем данные из таблицы
cities = []
for row in table.find_all('tr')[1:]:  # Пропускаем заголовок таблицы
    cols = row.find_all('td')
    if len(cols) > 1:
        city = cols[1].text.strip()
        cities.append(city)

print(cities)

['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Самара', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград']


In [ ]:
# текст запроса

cities_str = ', '.join(f"'{city}'" for city in cities)

query_5_7 = f'''
    WITH x AS ( 
        SELECT
            a.name as name,
            COUNT(v.id) AS cnt
        FROM public.vacancies AS v
        JOIN public.employers AS e ON v.employer_id = e.id
        JOIN public.areas AS a ON v.area_id = a.id
        WHERE e.name = 'Яндекс' AND a.name IN ({cities_str})
        GROUP BY a.name
    )
    SELECT
        name,
        cnt
    FROM x
    UNION ALL
    SELECT
        'Total' AS name,
        SUM(cnt) AS cnt
    FROM x
    '''

In [ ]:
# результат запроса

df = pd.read_sql_query(query_5_7, connection)
print(f'Количество строк в выводе: {df.shape[0]}')
df


Количество строк в выводе: 17


,name,cnt
0,Волгоград,24.0
1,Воронеж,32.0
2,Екатеринбург,39.0
3,Казань,25.0
4,Краснодар,30.0
5,Красноярск,23.0
6,Москва,54.0
7,Нижний Новгород,36.0
8,Новосибирск,35.0
9,Омск,21.0


***

### Выводы по анализу работодателей

* Яндекс является компанией, которая разместила наибольшее количество вакансий - **1933**;
* **В регионе 'Россия' наибольшое количество работодателей (410)**, которые не разместили вакансии;
* **Яндекс** является компанией, разместившей **наибольшое количество вакансией в разных регионах: 181**;
* **8419 компаний, не предоставляют информацию, в какой сфере дейтельности они работают**, что составляет **36%** от общего количества работодателей;
* **15% компаний в базе**, занимаются разработкой программного обеспечения;
* **485 вакансий, разместила компаний Яндекс в городах милионников**, что составляет **25%** от общего количества вакансий компании; 
* **1% вакансий в Москве**, открыто Яндексом


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса

query_6_1 = f'''
        SELECT
            COUNT(v.id)
        FROM public.vacancies as v
        WHERE lower(v.name) LIKE '%data%' OR lower(v.name) LIKE '%данн%'
        '''

In [ ]:
# результат запроса

df = pd.read_sql_query(query_6_1, connection)
print(f'{df['count'][0]} Вакансий имеет отношение к данным в нашей базе')

1771 Вакансий имеет отношение к данным в нашей базе


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# текст запроса

query_6_2 = f'''
    SELECT
        COUNT(*)
    FROM public.vacancies as v
    WHERE 
        (
            v.name ILIKE '%data scientist%' OR
            v.name ILIKE '%data science%' OR
            v.name ILIKE '%исследователь данных%' OR
            (v.name ILIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
            v.name ILIKE '%machine learning%' OR
            v.name ILIKE '%машинн%обучен%'
        )
        AND
        (
            v.name ILIKE '%junior%' OR
            v.experience = 'Нет опыта' OR
            v.employment = 'Стажировка'
        )
    '''


In [ ]:
# результат запроса

df = pd.read_sql_query(query_6_2, connection)
print(f'{df['count'][0]} подходящих вакансий для начинающего дата-сайентиста')

51 подходящих вакансий для начинающего дата-сайентиста


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса

query_6_3 = f'''
        SELECT
            COUNT(v.id)
        FROM public.vacancies AS v
        WHERE 
            (
                v.name ILIKE '%data scientist%' OR
                v.name ILIKE '%data science%' OR
                v.name ILIKE '%исследователь данных%' OR
                (v.name ILIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
                v.name ILIKE '%machine learning%' OR
                v.name ILIKE '%машинн%обучен%'
            )
            AND
            (
                v.key_skills ILIKE '%SQL%' OR
                v.key_skills ILIKE '%postgres%'
            )
'''

In [ ]:
# результат запроса

df = pd.read_sql_query(query_6_3, connection)
print(f'{df['count'][0]} вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres')

229 вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса

query_6_4 = f'''
        SELECT
            COUNT(v.id)
        FROM public.vacancies AS v
        WHERE 
            (
                v.name ILIKE '%data scientist%' OR
                v.name ILIKE '%data science%' OR
                v.name ILIKE '%исследователь данных%' OR
                (v.name ILIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
                v.name ILIKE '%machine learning%' OR
                v.name ILIKE '%машинн%обучен%'
            )
            AND
                v.key_skills ILIKE '%python%'
'''


In [ ]:
# результат запроса

df = pd.read_sql_query(query_6_4, connection)
print(f'{df['count'][0]} количество вакансий, в которых в качестве ключевого навыка указан Python')

357 количество вакансий, в которых в качестве ключевого навыка указан Python


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
# текст запроса

query_6_5 = f'''
        SELECT
            ROUND(AVG(LENGTH(v.key_skills) - LENGTH(REPLACE(v.key_skills, CHR(9),'')) + 1),2) as c
        FROM public.vacancies AS v
        WHERE 
            (
                v.name ILIKE '%data scientist%' OR
                v.name ILIKE '%data science%' OR
                v.name ILIKE '%исследователь данных%' OR
                (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR 
                v.name ILIKE '%machine learning%' OR
                v.name ILIKE '%машинн%обучен%'
            )
'''

In [ ]:
# результат запроса

df = pd.read_sql_query(query_6_5, connection)
print(f'{df['c'][0]} ключевых навыков в среднем указывают в вакансиях для DS')

6.41 ключевых навыков в среднем указывают в вакансиях для DS


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
# текст запроса

query_6_6 = f'''
    SELECT
        v.experience,
        ROUND(AVG(
            CASE
                WHEN v.salary_from IS NOT NULL AND v.salary_to IS NOT NULL THEN (v.salary_from + v.salary_to) / 2
                WHEN v.salary_from IS NOT NULL THEN v.salary_from
                WHEN v.salary_to IS NOT NULL THEN v.salary_to
            END
        )) as average_salary
    FROM public.vacancies AS v
    WHERE 
        (
            v.name ILIKE '%data scientist%' OR
            v.name ILIKE '%data science%' OR
            v.name ILIKE '%исследователь данных%' OR
            (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%') OR
            v.name ILIKE '%machine learning%' OR
            v.name ILIKE '%машинн%обучен%'
        )
        AND (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
    GROUP BY v.experience
    --HAVING v.experience = 'От 3 до 6 лет'
'''

In [ ]:
# результат запроса

df = pd.read_sql_query(query_6_6, connection)
df

,experience,average_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

### Выводы по предметному анализу

* **1771 Вакансий имеет отношение к данным** в нашей базе, что составляет **8%** от общего количества;
* **51 подходящих вакансий для начинающего дата-сайентиста**
* **229 вакансий для DS**, в которых в качестве ключевого навыка указан **SQL или postgres**;
* **357 количество вакансий**, в которых в качестве ключевого навыка указан **Python**;
* **6.41 ключевых навыков в среднем** указывают в вакансиях для DS;
* В **3.25 раза** средняя з/п для кандидата с опытом от 3 лет выше чем для соискателей без опыта;


# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

### Общие выводы по проекту

#### Общие инфо по датасету

* В данном датасете много вакансий из сферы IT, либо смежных сфер; 
* В датасете можно встретить вакансий из почти каждого региона, а также других стран, при этом вакансии из других стран имеет более выгодные условия, но их мало;
* В датасете много потенциальных выбросов, это можно заметить по данным з/п. Для дальнейшей работы, возможно стоит произвести отчистку данных; 

#### Прогнозы и тенденции

* Вакансии, связанные с технологиями и данными, вероятно, будут оставаться популярными, учитывая текущие тренды в цифровой трансформации;
* Зарплаты для специалистов с опытом, скорее всего, будут расти, особенно в высокотехнологичных и востребованных областях;
* Города с развитой инфраструктурой и экономикой будут продолжать привлекать больше вакансий и предлагать более высокие зарплаты.
* Сферы, связанные с ИТ и инновациями, будут оставаться на передовой, что подтверждается большим количеством вакансий в этих отраслях.




In [ ]:
# Топ 10 вакансий в нашей базе

query_7_1 = f'''
    SELECT
    v.name,
    COUNT(v.id) AS vacancy_count
FROM public.vacancies AS v
GROUP BY v.name
ORDER BY vacancy_count DESC
LIMIT 10
'''

df = pd.read_sql_query(query_7_1, connection)
df

,name,vacancy_count
0,Системный администратор,1761
1,Программист 1С,1153
2,Бизнес-аналитик,488
3,Специалист технической поддержки,457
4,Системный аналитик,432
5,Аналитик,404
6,Инженер-программист,389
7,Программист 1C,360
8,Frontend-разработчик,242
9,Инженер технической поддержки,230


In [ ]:
# Распределение средней з/п в разделе требуемого опыта

query_7_2 = f'''
SELECT
    v.experience,
    ROUND(AVG(
        CASE
            WHEN v.salary_from IS NOT NULL AND v.salary_to IS NOT NULL THEN (v.salary_from + v.salary_to) / 2
            WHEN v.salary_from IS NOT NULL THEN v.salary_from
            WHEN v.salary_to IS NOT NULL THEN v.salary_to
        END
    )) AS average_salary
FROM public.vacancies AS v
WHERE (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
GROUP BY v.experience
ORDER BY average_salary DESC
'''

df = pd.read_sql_query(query_7_2, connection)
df

,experience,average_salary
0,Более 6 лет,166256.0
1,От 3 до 6 лет,133211.0
2,От 1 года до 3 лет,76541.0
3,Нет опыта,40472.0


In [ ]:
# Топ-10 средней з/п по распределению по регионам

query_7_3 = f'''
SELECT
    a.name AS area,
    ROUND(AVG(
        CASE
            WHEN v.salary_from IS NOT NULL AND v.salary_to IS NOT NULL THEN (v.salary_from + v.salary_to) / 2
            WHEN v.salary_from IS NOT NULL THEN v.salary_from
            WHEN v.salary_to IS NOT NULL THEN v.salary_to
        END
    )) AS average_salary
FROM public.vacancies AS v
JOIN public.areas AS a ON v.area_id = a.id
WHERE (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
GROUP BY a.name
ORDER BY average_salary DESC
LIMIT 10
'''

df = pd.read_sql_query(query_7_3, connection)
df

,area,average_salary
0,Германия,452751.0
1,Болгария,324348.0
2,Испания,300000.0
3,Черногория,288589.0
4,Литва,263019.0
5,Польша,257156.0
6,Петропавловка (Республика Бурятия),250000.0
7,Турция,235435.0
8,Кипр,223736.0
9,Кутаиси,220475.0


In [ ]:
# Топ-10 сфер детельности по количеству вакансий

query_7_4 = f'''
SELECT
    i.name AS industry,
    COUNT(v.id) AS vacancy_count
FROM public.vacancies AS v
JOIN public.employers AS e ON v.employer_id = e.id
JOIN public.employers_industries AS ei ON e.id = ei.employer_id
JOIN public.industries AS i ON ei.industry_id = i.id
GROUP BY i.name
ORDER BY vacancy_count DESC
LIMIT 10
'''

df = pd.read_sql_query(query_7_4, connection)
df

,industry,vacancy_count
0,Разработка программного обеспечения,12499
1,"Системная интеграция, автоматизации технологи...",11034
2,"Интернет-компания (поисковики, платежные систе...",6413
3,Банк,2742
4,Мобильная связь,1585
5,Кадровые агентства,1143
6,Консалтинговые услуги,1137
7,Интернет-провайдер,1095
8,Розничная сеть (продуктовая),1034
9,Фиксированная связь,1028


In [ ]:
# Топ 10 сфер деятельности по средней заработной плате

query_7_5 = f'''
SELECT
    i.name AS industry,
    ROUND(AVG(
        CASE
            WHEN v.salary_from IS NOT NULL AND v.salary_to IS NOT NULL THEN (v.salary_from + v.salary_to) / 2
            WHEN v.salary_from IS NOT NULL THEN v.salary_from
            WHEN v.salary_to IS NOT NULL THEN v.salary_to
        END
    )) AS average_salary
FROM public.vacancies AS v
JOIN public.employers AS e ON v.employer_id = e.id
JOIN public.employers_industries AS ei ON e.id = ei.employer_id
JOIN public.industries AS i ON ei.industry_id = i.id
WHERE (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
GROUP BY i.name
ORDER BY average_salary DESC
LIMIT 10
'''

df = pd.read_sql_query(query_7_5, connection)
df

,industry,average_salary
0,"Агрохимия (продвижение, оптовая торговля)",381429.0
1,Агрохимия (производство),339875.0
2,Игорный бизнес,191905.0
3,Корма для животных (производство),168300.0
4,Зоомагазин,167613.0
5,Кадровые агентства,144530.0
6,"Лакокрасочная продукция, сырье (производство)",142803.0
7,"Бытовая химия, парфюмерия, косметика (произво...",130120.0
8,"Лакокрасочная продукция, сырье (продвижение, о...",128445.0
9,"Дорожно-строительная техника, сельскохозяйстве...",122878.0


In [ ]:

# Количество вакансий по ключевым навыков встречающихся в них

query_7_6 = f'''
SELECT
    unnest(string_to_array(v.key_skills, CHR(9))) AS skill,
    COUNT(v.id) AS vacancy_count
FROM public.vacancies AS v
GROUP BY skill
ORDER BY vacancy_count DESC
LIMIT 10
'''

df = pd.read_sql_query(query_7_6, connection)
df



,skill,vacancy_count
0,SQL,6977
1,Git,4731
2,Английский язык,4525
3,Работа в команде,3923
4,Linux,3747
5,JavaScript,3625
6,Python,3445
7,Управление проектами,2764
8,1С программирование,2647
9,PostgreSQL,2452


In [ ]:
# Топ 10 работодателей по средней предлогаемой з/п

query_7_7 = f'''

SELECT
    e.name AS employer_name,
    ROUND(AVG(
        CASE
            WHEN v.salary_from IS NOT NULL AND v.salary_to IS NOT NULL THEN (v.salary_from + v.salary_to) / 2
            WHEN v.salary_from IS NOT NULL THEN v.salary_from
            WHEN v.salary_to IS NOT NULL THEN v.salary_to
        END
    )) AS average_salary
FROM public.vacancies AS v
JOIN public.employers AS e ON v.employer_id = e.id
WHERE (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
GROUP BY e.name
ORDER BY average_salary DESC
LIMIT 10
'''

df = pd.read_sql_query(query_7_7, connection)
df



,employer_name,average_salary
0,Alpha Personnel,1000000.0
1,ЦТП,650000.0
2,RuWork,555261.0
3,Петухова Алия,552700.0
4,Агентство недвижимости ЛЕТО,550000.0
5,Пикабу,550000.0
6,BeRecruited,550000.0
7,ВСП Партнер,500000.0
8,"Август, Фирма",500000.0
9,StaffRecruitment,483333.0
